In [1]:
import http.client
import urllib.parse

In [8]:
conn = http.client.HTTPSConnection("twitter241.p.rapidapi.com")

headers = {
    'x-rapidapi-key': "7ee1c65373msh181b4a01f21d239p1e5ff7jsnc2f07e071e6d",
    'x-rapidapi-host': "twitter241.p.rapidapi.com"
}

query_params = {
    'type': 'Top',
    'count': 5,
    'query': "apple company's CEO latest tweets"
}
url_safe_query = urllib.parse.urlencode(query_params)
url_safe_query

'type=Top&count=5&query=apple+company%27s+CEO+latest+tweets'

In [9]:
conn.request("GET", f"/search-v2?{url_safe_query}", headers=headers)

In [10]:
res = conn.getresponse()
data = res.read()

In [11]:
import json
output_dict = json.loads(data)

In [6]:
output_dict

{'cursor': {'bottom': 'DAACCgACGsqd3_HAJxAKAAMayp3f8b_Y8AgABAAAAAILAAUAAADoRW1QQzZ3QUFBZlEvZ0dKTjB2R3AvQUFBQUJNV01wOTd1SlFBQUJZcXc0U0VXaEFCR1hEUHcxYVdVZDBUOUYzK2wxZXdCUmVXbkxuMjE0RkJGaU9Zc3BUWDBBRVhxODJ3alJhZ0RSZWQ4QklqbTRHTUdVUGI5TzJhc0ZvVXpMUmNOQlJRQVJYYmd0QnhHakFBRktKSVhpaFhFQUVKdDQxT0JSYmdBQWZIUjc0c1FpQUJDMDhzWUkvV2dBQVZ3TVUyOGxmUUJCWXEySkt5bWpBQkZqR2Y4SzJWVUFJVDVWTlhaaGNRQVE9PQgABgAAAAAIAAcAAAAADAAICgABB8dHvixCIAEAAAA',
  'top': 'DAACCgACGsqd3_HAJxAKAAMayp3f8b_Y8AgABAAAAAELAAUAAADoRW1QQzZ3QUFBZlEvZ0dKTjB2R3AvQUFBQUJNV01wOTd1SlFBQUJZcXc0U0VXaEFCR1hEUHcxYVdVZDBUOUYzK2wxZXdCUmVXbkxuMjE0RkJGaU9Zc3BUWDBBRVhxODJ3alJhZ0RSZWQ4QklqbTRHTUdVUGI5TzJhc0ZvVXpMUmNOQlJRQVJYYmd0QnhHakFBRktKSVhpaFhFQUVKdDQxT0JSYmdBQWZIUjc0c1FpQUJDMDhzWUkvV2dBQVZ3TVUyOGxmUUJCWXEySkt5bWpBQkZqR2Y4SzJWVUFJVDVWTlhaaGNRQVE9PQgABgAAAAAIAAcAAAAADAAICgABB8dHvixCIAEAAAA'},
 'result': {'timeline': {'instructions': [{'type': 'TimelineAddEntries',
     'entries': [{'entryId': 'toptabsrpusermodule-1930528975440838656',
       's

In [23]:
def extract_valuable_information(data):
    """
    Extracts all valuable information from a nested data structure.

    Args:
        data (dict): The input data structure (e.g., JSON-like dictionary).

    Returns:
        dict: A dictionary containing extracted information.
    """
    extracted_data = {
        "users": [],
        "tweets": [],
        "feedback": []
    }

    # Extract user information
    instructions = data.get('result', {}).get('timeline', {}).get('instructions', [])
    for instruction in instructions:
        entries = instruction.get('entries', [])
        for entry in entries:
            content = entry.get('content', {})
            if content.get('entryType') == 'TimelineTimelineItem':
                item_content = content.get('itemContent', {})
                if item_content.get('itemType') == 'TimelineTweet':
                    tweet_result = item_content.get('tweet_results', {}).get('result', {})
                    tweet_legacy = tweet_result.get('legacy', {})
                    user_result = tweet_result.get('core', {}).get('user_results', {}).get('result', {})
                    user_legacy = user_result.get('legacy', {})

                    # Extract tweet information
                    tweet_info = {
                        "tweet_id": tweet_legacy.get('id_str', 'N/A'),
                        "text": tweet_legacy.get('full_text', 'N/A'),
                        "created_at": tweet_legacy.get('created_at', 'N/A'),
                        "hashtags": [hashtag.get('text') for hashtag in tweet_legacy.get('entities', {}).get('hashtags', [])],
                        "mentions": [mention.get('screen_name') for mention in tweet_legacy.get('entities', {}).get('user_mentions', [])],
                        "urls": [url.get('expanded_url') for url in tweet_legacy.get('entities', {}).get('urls', [])],
                        "media": [media.get('media_url_https') for media in tweet_legacy.get('entities', {}).get('media', [])],
                        "retweet_count": tweet_legacy.get('retweet_count', 0),
                        "favorite_count": tweet_legacy.get('favorite_count', 0),
                        "lang": tweet_legacy.get('lang', 'N/A')
                    }
                    extracted_data["tweets"].append(tweet_info)

                    # Extract user information
                    user_info = {
                        "user_id": user_legacy.get('id_str', 'N/A'),
                        "name": user_legacy.get('name', 'N/A'),
                        "screen_name": user_legacy.get('screen_name', 'N/A'),
                        "description": user_legacy.get('description', 'N/A'),
                        "followers_count": user_legacy.get('followers_count', 0),
                        "friends_count": user_legacy.get('friends_count', 0),
                        "statuses_count": user_legacy.get('statuses_count', 0),
                        "verified": user_legacy.get('verified', False),
                        "profile_image_url": user_legacy.get('profile_image_url_https', 'N/A'),
                        "created_at": user_legacy.get('created_at', 'N/A')
                    }
                    extracted_data["users"].append(user_info)

    # Extract feedback information
    feedback_actions = data.get('result', {}).get('timeline', {}).get('responseObjects', {}).get('feedbackActions', [])
    for feedback in feedback_actions:
        feedback_value = feedback.get('value', {})
        feedback_info = {
            "feedback_type": feedback_value.get('feedbackType', 'N/A'),
            "prompt": feedback_value.get('prompt', 'N/A'),
            "confirmation": feedback_value.get('confirmation', 'N/A')
        }
        extracted_data["feedback"].append(feedback_info)

    return extracted_data


# Example usage
result = extract_valuable_information(output_dict)
print(json.dumps(result, indent=4))

{
    "users": [
        {
            "user_id": "N/A",
            "name": "Thomas George",
            "screen_name": "thomasg_grizzle",
            "description": "Disruption, Growth, Energy & Mining. PM of the $DARP ETF. Former Head of Commodities at TD Asset Management. Spice King @grizzlemedia \ud83d\udc51\ud83c\udf36",
            "followers_count": 21430,
            "friends_count": 3936,
            "statuses_count": 48765,
            "verified": false,
            "profile_image_url": "https://pbs.twimg.com/profile_images/1675429884003229696/x5cFfA4J_normal.jpg",
            "created_at": "Tue Jan 30 15:54:09 +0000 2018"
        },
        {
            "user_id": "N/A",
            "name": "Greg Sarafan\u2122",
            "screen_name": "GSarafan",
            "description": "litigator, @HeadCountOrg Regional Coordinator,\ud83c\udf32\ud83d\udc7d\ud83c\udf32 #B4L \u2022 \u2b55 \u2022 \ud83d\udc80\ud83c\udf39\u26a1\ufe0f \u2022 #StarWars \u2022 \ud83c\udff3\ufe0f\u200d\ud8

In [34]:
def extract_tweets(data):
    """
    Extracts tweets from a nested data structure and enriches the dataset with additional fields.
    Args:
        data (dict): The input data structure (e.g., JSON-like dictionary).
    Returns:
        list: A list of dictionaries containing enriched tweet information.
    """
    tweets = []
    # Navigate through the timeline instructions
    instructions = data.get('result', {}).get('timeline', {}).get('instructions', [])
    for instruction in instructions:
        entries = instruction.get('entries', [])
        for entry in entries:
            content = entry.get('content', {})
            if content.get('entryType') == 'TimelineTimelineItem':
                item_content = content.get('itemContent', {})
                if item_content.get('itemType') == 'TimelineTweet':
                    tweet_result = item_content.get('tweet_results', {}).get('result', {})
                    tweet_legacy = tweet_result.get('legacy', {})
                    user_result = tweet_result.get('core', {}).get('user_results', {}).get('result', {})
                    user_legacy = user_result.get('legacy', {})

                    # Extract tweet information
                    tweet_info = {
                        "tweet_id": tweet_legacy.get('id_str', 'N/A'),
                        "text": tweet_legacy.get('full_text', 'N/A'),
                        'type': type(tweet_legacy.get('full_text', 'N/A')),
                        "created_at": tweet_legacy.get('created_at', 'N/A'),
                        "hashtags": [hashtag.get('text') for hashtag in tweet_legacy.get('entities', {}).get('hashtags', [])],
                        "mentions": [mention.get('screen_name') for mention in tweet_legacy.get('entities', {}).get('user_mentions', [])],
                        "urls": [url.get('expanded_url') for url in tweet_legacy.get('entities', {}).get('urls', [])],
                        "media": [media.get('media_url_https') for media in tweet_legacy.get('entities', {}).get('media', [])],
                        "retweet_count": tweet_legacy.get('retweet_count', 0),
                        "favorite_count": tweet_legacy.get('favorite_count', 0),
                        "lang": tweet_legacy.get('lang', 'N/A'),
                        "source": tweet_legacy.get('source', 'N/A'),
                        "is_quote_status": tweet_legacy.get('is_quote_status', False),
                        "quote_count": tweet_legacy.get('quote_count', 0),
                        "reply_count": tweet_legacy.get('reply_count', 0),
                        "user_id": user_legacy.get('id_str', 'N/A'),
                        "user_screen_name": user_legacy.get('screen_name', 'N/A'),
                        "user_name": user_legacy.get('name', 'N/A'),
                        "user_verified": user_legacy.get('verified', False),
                        "user_followers_count": user_legacy.get('followers_count', 0),
                        "user_profile_image": user_legacy.get('profile_image_url_https', 'N/A'),
                    }
                    tweets.append(tweet_info)
    return tweets
# Example usage
result = extract_tweets(output_dict)
# print(json.dumps(result, indent=4))

In [35]:
from pprint import pprint
pprint(result)

[{'created_at': 'Sun Oct 30 11:17:23 +0000 2022',
  'favorite_count': 4145,
  'hashtags': [],
  'is_quote_status': False,
  'lang': 'en',
  'media': [],
  'mentions': [],
  'quote_count': 168,
  'reply_count': 48,
  'retweet_count': 544,
  'source': 'N/A',
  'text': 'Apple CFO Luca Maestri out here making capital management sound '
          'like fine Italian arts 🤌🏾 $AAPL just built different \n'
          '\n'
          ' https://t.co/1HoIibDiLz',
  'tweet_id': '1586678648731467778',
  'type': <class 'str'>,
  'urls': ['https://x.com/ctbeiser/status/1586447808113975296/video/1'],
  'user_followers_count': 21430,
  'user_id': 'N/A',
  'user_name': 'Thomas George',
  'user_profile_image': 'https://pbs.twimg.com/profile_images/1675429884003229696/x5cFfA4J_normal.jpg',
  'user_screen_name': 'thomasg_grizzle',
  'user_verified': False},
 {'created_at': 'Thu Mar 07 00:00:11 +0000 2019',
  'favorite_count': 134,
  'hashtags': [],
  'is_quote_status': False,
  'lang': 'en',
  'media': ['htt

In [31]:
ans = extract_valuable_information(output_dict)

In [32]:
ans

{'users': [{'user_id': 'N/A',
   'name': 'Thomas George',
   'screen_name': 'thomasg_grizzle',
   'description': 'Disruption, Growth, Energy & Mining. PM of the $DARP ETF. Former Head of Commodities at TD Asset Management. Spice King @grizzlemedia 👑🌶',
   'followers_count': 21430,
   'friends_count': 3936,
   'statuses_count': 48765,
   'verified': False,
   'profile_image_url': 'https://pbs.twimg.com/profile_images/1675429884003229696/x5cFfA4J_normal.jpg',
   'created_at': 'Tue Jan 30 15:54:09 +0000 2018'},
  {'user_id': 'N/A',
   'name': 'Greg Sarafan™',
   'screen_name': 'GSarafan',
   'description': 'litigator, @HeadCountOrg Regional Coordinator,🌲👽🌲 #B4L • ⭕ • 💀🌹⚡️ • #StarWars • 🏳️\u200d🌈 • CatDad 🐈\u200d⬛️ • PlantDad 🪴',
   'followers_count': 23588,
   'friends_count': 1274,
   'statuses_count': 79299,
   'verified': False,
   'profile_image_url': 'https://pbs.twimg.com/profile_images/1900994775496425472/bQLCRYmJ_normal.jpg',
   'created_at': 'Tue May 12 21:41:28 +0000 2009'},
  {

## Twitter API

In [39]:
import http.client
import urllib.parse
conn = http.client.HTTPSConnection("twitter-api45.p.rapidapi.com")

headers = {
    'x-rapidapi-key': "7ee1c65373msh181b4a01f21d239p1e5ff7jsnc2f07e071e6d",
    'x-rapidapi-host': "twitter-api45.p.rapidapi.com"
}
query_params = {
    "query": "apple ceo's latest tweets",
    "search_type": "Top"
}
# get url safe query
url_safe_query = urllib.parse.urlencode(query_params)
url_safe_query

'query=apple+ceo%27s+latest+tweets&search_type=Top'

In [40]:
conn.request("GET", f"/search.php?{url_safe_query}", headers=headers)

In [41]:
res = conn.getresponse()
data = res.read()

In [42]:
output_dict = json.loads(data)
output_dict

{'status': 'ok',
 'timeline': [{'type': 'tweet',
   'tweet_id': '1771815471832027387',
   'screen_name': 'HuXijin_GT',
   'bookmarks': 20,
   'favorites': 214,
   'created_at': 'Sun Mar 24 08:25:01 +0000 2024',
   'text': '“I love China🇨🇳 and the [Chinese] people,” said Apple CEO @tim_cook in Beijing today. https://t.co/TkEsf44H4d',
   'lang': 'en',
   'source': '<a href="https://mobile.twitter.com" rel="nofollow">Twitter Web App</a>',
   'quotes': 16,
   'replies': 145,
   'conversation_id': '1771815471832027387',
   'retweets': 49,
   'views': '46996',
   'entities': {'hashtags': [],
    'media': [{'display_url': 'pic.x.com/TkEsf44H4d',
      'expanded_url': 'https://x.com/HuXijin_GT/status/1771815471832027387/video/1',
      'id_str': '1771815351593873408',
      'indices': [86, 109],
      'media_key': '7_1771815351593873408',
      'media_url_https': 'https://pbs.twimg.com/ext_tw_video_thumb/1771815351593873408/pu/img/LaxvkinxM35agsdP.jpg',
      'type': 'video',
      'url': 'htt

In [46]:
def extract_rich_tweet_data(data):
    rich_data = []
    # traverse  through the timeline to extract tweets
    for tweet in data.get("timeline", []):
        if tweet.get("type") == "tweet":
            # extract tweet information
            tweet_info = {
                "tweet_id": tweet.get("tweet_id", ""),
                "text": tweet.get("text", ""),
                "created_at": tweet.get("created_at", ""),
                "screen_name": tweet.get("screen_name", ""),
                "source": tweet.get("source", ""),
                "lang": tweet.get("lang", ""),
                "conversation_id": tweet.get("conversation_id", ""),
                "retweets": tweet.get("retweets", 0),
                "replies": tweet.get("replies", 0),
                "quotes": tweet.get("quotes", 0),
                "favorites": tweet.get("favorites", 0),
                "bookmarks": tweet.get("bookmarks", 0),
                "views": tweet.get("views", "0"),
                "hashtags": [hashtag.get("text") for hashtag in tweet.get("entities", {}).get("hashtags", [])],
                "mentions": [
                    {
                        "id": mention.get("id_str", ""),
                        "name": mention.get("name", ""),
                        "screen_name": mention.get("screen_name", ""),
                    }
                    for mention in tweet.get("entities", {}).get("user_mentions", [])
                ],
                "urls": [
                    {
                        "url": url.get("url", ""),
                        "expanded_url": url.get("expanded_url", ""),
                        "display_url": url.get("display_url", ""),
                    }
                    for url in tweet.get("entities", {}).get("urls", [])
                ],
                "media": [
                    {
                        "type": media.get("type", ""),
                        "url": media.get("media_url_https", ""),
                        "video_info": media.get("video_info", {}),
                        "sizes": media.get("sizes", {}),
                    }
                    for media in tweet.get("entities", {}).get("media", [])
                ],
                "user_info": {
                    "screen_name": tweet.get("user_info", {}).get("screen_name", ""),
                    "name": tweet.get("user_info", {}).get("name", ""),
                    "description": tweet.get("user_info", {}).get("description", ""),
                    "followers_count": tweet.get("user_info", {}).get("followers_count", 0),
                    "friends_count": tweet.get("user_info", {}).get("friends_count", 0),
                    "verified": tweet.get("user_info", {}).get("verified", False),
                    "avatar": tweet.get("user_info", {}).get("avatar", ""),
                },
            }
            rich_data.append(tweet_info)

    return rich_data

In [44]:
ans = extract_rich_tweet_data(output_dict)

In [45]:
ans

[{'tweet_id': '1771815471832027387',
  'text': '“I love China🇨🇳 and the [Chinese] people,” said Apple CEO @tim_cook in Beijing today. https://t.co/TkEsf44H4d',
  'created_at': 'Sun Mar 24 08:25:01 +0000 2024',
  'screen_name': 'HuXijin_GT',
  'source': '<a href="https://mobile.twitter.com" rel="nofollow">Twitter Web App</a>',
  'lang': 'en',
  'conversation_id': '1771815471832027387',
  'retweets': 49,
  'replies': 145,
  'quotes': 16,
  'favorites': 214,
  'bookmarks': 20,
  'views': '46996',
  'hashtags': [],
  'mentions': [{'id': '1636590253',
    'name': 'Tim Cook',
    'screen_name': 'tim_cook'}],
  'urls': [],
  'media': [{'type': 'video',
    'url': 'https://pbs.twimg.com/ext_tw_video_thumb/1771815351593873408/pu/img/LaxvkinxM35agsdP.jpg',
    'video_info': {'aspect_ratio': [17, 30],
     'duration_millis': 6520,
     'variants': [{'content_type': 'application/x-mpegURL',
       'url': 'https://video.twimg.com/ext_tw_video/1771815351593873408/pu/pl/kCFgaMGM8YxvJfHG.m3u8?tag=12'}

## Twitter AIO

In [105]:
import http.client
import urllib.parse
from datetime import date, timedelta
conn = http.client.HTTPSConnection("twitter-aio.p.rapidapi.com")

headers = {
    'x-rapidapi-key': "7ee1c65373msh181b4a01f21d239p1e5ff7jsnc2f07e071e6d",
    'x-rapidapi-host': "twitter-aio.p.rapidapi.com"
}

filters =  {
    "lang": "en",
    "since": (date.today() - timedelta(days=30)).isoformat(),
    "until": date.today().isoformat()   
}
query_params = {
    'count': 5,
    'category': 'Latest',
    'filters': json.dumps(filters),
    'includeTimestamp': 'true',
}
url_safe_query = urllib.parse.urlencode(query_params)
url_safe_query

'count=5&category=Latest&filters=%7B%22lang%22%3A+%22en%22%2C+%22since%22%3A+%222025-05-06%22%2C+%22until%22%3A+%222025-06-05%22%7D&includeTimestamp=true'

In [110]:
# since%22%3A%20%222020-10-01%22%7D
conn.request("GET", f"/search/apple+ceo?{url_safe_query}", headers=headers)
# conn.request("GET", f"/search/{url_safe_query}", headers=headers)

In [111]:
res = conn.getresponse()
data = res.read()

In [112]:
output_dict = json.loads(data)

In [120]:
output_dict.get("entries", [])

[{'type': 'TimelineAddEntries',
  'entries': [{'entryId': 'tweet-1930407437904773542',
    'sortIndex': '1930618769344823296',
    'content': {'entryType': 'TimelineTimelineItem',
     '__typename': 'TimelineTimelineItem',
     'itemContent': {'itemType': 'TimelineTweet',
      '__typename': 'TimelineTweet',
      'tweet_results': {'result': {'__typename': 'Tweet',
        'rest_id': '1930407437904773542',
        'core': {'user_results': {'result': {'__typename': 'User',
           'id': 'VXNlcjo2MjkwNzcyMw==',
           'rest_id': '62907723',
           'affiliates_highlighted_label': {},
           'has_graduated_access': True,
           'is_blue_verified': False,
           'legacy': {'following': False,
            'can_dm': True,
            'can_media_tag': False,
            'created_at': 'Tue Aug 04 19:41:53 +0000 2009',
            'default_profile': False,
            'default_profile_image': False,
            'description': 'Keeping things simple by doing what I love. Te

In [61]:
from datetime import date
print(date.today())

2025-06-05


In [63]:
type(date.today().isoformat())

str

In [121]:
def extract_rich_tweet_data(data):
    """
    Extracts valuable information from a dataset of tweets and returns a rich dataset.

    Args:
        data (dict): The input data structure containing tweets.

    Returns:
        list: A list of dictionaries containing enriched tweet information.
    """
    rich_data = []
    # get the current year
    current_year = datetime.now().year
    
    # Navigate through the timeline entries
    entries = data.get("entries", [])
    for entry in entries:
        
        # go to the nested entries section
        nested_entries = entry.get("entries", {})
        for nested_entry in nested_entries:
            content = nested_entry.get("content", {})
            if content.get("entryType") == "TimelineTimelineItem":
                item_content = content.get("itemContent", {})
                if item_content.get("itemType") == "TimelineTweet":
                    tweet_result = item_content.get("tweet_results", {}).get("result", {})
                    tweet_legacy = tweet_result.get("legacy", {})
                    user_result = tweet_result.get("core", {}).get("user_results", {}).get("result", {})
                    user_legacy = user_result.get("legacy", {})

                    # get created_at
                    created_at = tweet_legacy.get("created_at", "")
                    # filter tweets based on date of creation
                    tweet_date = datetime.strptime(created_at, "%a %b %d %H:%M:%S %z %Y")
                    if(tweet_date.year < current_year):
                        continue
                    # Extract tweet information
                    tweet_info = {
                        "tweet_id": tweet_legacy.get("id_str", ""),
                        "text": tweet_legacy.get("full_text", ""),
                        "created_at": created_at,
                        "conversation_id": tweet_legacy.get("conversation_id_str", ""),
                        "retweets": tweet_legacy.get("retweet_count", 0),
                        "replies": tweet_legacy.get("reply_count", 0),
                        "quotes": tweet_legacy.get("quote_count", 0),
                        "favorites": tweet_legacy.get("favorite_count", 0),
                        "lang": tweet_legacy.get("lang", ""),
                        "source": tweet_legacy.get("source", ""),
                        "hashtags": [hashtag.get("text") for hashtag in tweet_legacy.get("entities", {}).get("hashtags", [])],
                        "mentions": [
                            {
                                "id": mention.get("id_str", ""),
                                "name": mention.get("name", ""),
                                "screen_name": mention.get("screen_name", ""),
                            }
                            for mention in tweet_legacy.get("entities", {}).get("user_mentions", [])
                        ],
                        "urls": [
                            {
                                "url": url.get("url", ""),
                                "expanded_url": url.get("expanded_url", ""),
                                "display_url": url.get("display_url", ""),
                            }
                            for url in tweet_legacy.get("entities", {}).get("urls", [])
                        ],
                        "media": [
                            {
                                "type": media.get("type", ""),
                                "url": media.get("media_url_https", ""),
                                "sizes": media.get("sizes", {}),
                            }
                            for media in tweet_legacy.get("extended_entities", {}).get("media", [])
                        ],
                    }
    
                    # Extract user information
                    user_info = {
                        "user_id": user_legacy.get("id_str", ""),
                        "name": user_legacy.get("name", ""),
                        "screen_name": user_legacy.get("screen_name", ""),
                        "description": user_legacy.get("description", ""),
                        "followers_count": user_legacy.get("followers_count", 0),
                        "friends_count": user_legacy.get("friends_count", 0),
                        "statuses_count": user_legacy.get("statuses_count", 0),
                        "verified": user_legacy.get("verified", False),
                        "profile_image_url": user_legacy.get("profile_image_url_https", ""),
                        "profile_banner_url": user_legacy.get("profile_banner_url", ""),
                    }
    
                    # Combine tweet and user information
                    tweet_info["user_info"] = user_info
                    rich_data.append(tweet_info)

    return rich_data


# Example usage
if __name__ == "__main__":
    import json

    # Replace with your actual data
    with open("/home/devansh-rathore/Desktop/scraping/temp.txt", "r") as file:
        data = eval(file.read())  # Use eval to parse the data as a Python dictionary

    result = extract_rich_tweet_data(data)
    print(json.dumps(result, indent=4))

[
    {
        "tweet_id": "1930407437904773542",
        "text": "Qualcomm CEO discusses the company\u2019s post-Apple future https://t.co/WvrY47J6av",
        "created_at": "Wed Jun 04 23:32:53 +0000 2025",
        "conversation_id": "1930407437904773542",
        "retweets": 0,
        "replies": 0,
        "quotes": 0,
        "favorites": 0,
        "lang": "en",
        "source": "",
        "hashtags": [],
        "mentions": [],
        "urls": [],
        "media": [
            {
                "type": "photo",
                "url": "https://pbs.twimg.com/media/GsovVcQWwAEpqpT.jpg",
                "sizes": {
                    "large": {
                        "h": 680,
                        "w": 932,
                        "resize": "fit"
                    },
                    "medium": {
                        "h": 680,
                        "w": 932,
                        "resize": "fit"
                    },
                    "small": {
               

In [117]:
result

[]

## Using Old-Bird API

In [19]:
import http.client
from datetime import datetime, timedelta, date
import urllib.parse
conn = http.client.HTTPSConnection("twitter154.p.rapidapi.com")

headers = {
    'x-rapidapi-key': "7ee1c65373msh181b4a01f21d239p1e5ff7jsnc2f07e071e6d",
    'x-rapidapi-host': "twitter154.p.rapidapi.com"
}
current_year = datetime.now().year
current_date = date.today() # date time object
limiting_epoch = datetime.now().date().replace(day = 1, month = 1)
query_params = {
    'query': 'apple ceo',
    'section': 'top',
    'min_retweets': 1,
    'min_likes': 1,
    'limit': 5,
    'start_date': limiting_epoch.isoformat(),
    'language': 'en',
    'end_date': current_date.isoformat()
}
url_safe_query = urllib.parse.urlencode(query_params)
url_safe_query


'query=apple+ceo&section=top&min_retweets=1&min_likes=1&limit=5&start_date=2025-01-01&language=en&end_date=2025-06-08'

In [20]:
conn.request("GET", f"/search/search?{url_safe_query}", headers=headers)

In [21]:
res = conn.getresponse()
data = res.read()

In [23]:
import json
output_dict = json.loads(data)
output_dict

{'results': [{'tweet_id': '1931002435889254479',
   'creation_date': 'Fri Jun 06 14:57:11 +0000 2025',
   'text': 'On Monday, we sent a letter to Apple CEO Tim Cook, demanding an apology and the removal of a scene from one of its shows that involves "Eucharistic desecration." https://t.co/Gf137tSqoS',
   'media_url': ['https://pbs.twimg.com/media/GsxMezyWcAAv2xs.jpg',
    'https://pbs.twimg.com/media/GsxMezyXEAA48N0.jpg'],
   'video_url': None,
   'user': {'creation_date': 'Fri Dec 12 17:22:36 +0000 2008',
    'user_id': '18080620',
    'username': 'CatholicVote',
    'name': 'CatholicVote',
    'follower_count': 89245,
    'following_count': 2410,
    'favourites_count': 5528,
    'is_private': None,
    'is_verified': False,
    'is_blue_verified': False,
    'location': '',
    'profile_pic_url': 'https://pbs.twimg.com/profile_images/796404920392028160/T7ohIMTJ_normal.jpg',
    'profile_banner_url': 'https://pbs.twimg.com/profile_banners/18080620/1670947551',
    'description': 'Cat

In [24]:
def extract_rich_data(data):
    """
    Extracts valuable information from the provided data and returns a rich data dictionary.

    Args:
        data (dict): The input data structure containing tweets and user information.

    Returns:
        list: A list of dictionaries containing enriched tweet and user information.
    """
    rich_data = []

    # Iterate through the results
    for tweet in data.get("results", []):
        # Extract tweet information
        tweet_info = {
            "tweet_id": tweet.get("tweet_id", ""),
            "creation_date": tweet.get("creation_date", ""),
            "text": tweet.get("text", ""),
            "language": tweet.get("language", ""),
            "favorite_count": tweet.get("favorite_count", 0),
            "retweet_count": tweet.get("retweet_count", 0),
            "reply_count": tweet.get("reply_count", 0),
            "quote_count": tweet.get("quote_count", 0),
            "views": tweet.get("views", 0),
            "video_view_count": tweet.get("video_view_count", None),
            "source": tweet.get("source", ""),
            "conversation_id": tweet.get("conversation_id", ""),
            "in_reply_to_status_id": tweet.get("in_reply_to_status_id", None),
            "quoted_status_id": tweet.get("quoted_status_id", None),
            "expanded_url": tweet.get("expanded_url", ""),
            "media": [],
            "videos": [],
        }

        # Extract media information
        extended_entities = tweet.get("extended_entities", {}).get("media", [])
        for media in extended_entities:
            media_info = {
                "type": media.get("type", ""),
                "url": media.get("media_url_https", ""),
                "display_url": media.get("display_url", ""),
                "expanded_url": media.get("expanded_url", ""),
                "sizes": media.get("sizes", {}),
                "video_info": media.get("video_info", {}),
            }
            if media_info["type"] == "video":
                tweet_info["videos"].append(media_info)
            else:
                tweet_info["media"].append(media_info)

        # Extract user information
        user = tweet.get("user", {})
        user_info = {
            "user_id": user.get("user_id", ""),
            "username": user.get("username", ""),
            "name": user.get("name", ""),
            "creation_date": user.get("creation_date", ""),
            "follower_count": user.get("follower_count", 0),
            "following_count": user.get("following_count", 0),
            "favourites_count": user.get("favourites_count", 0),
            "listed_count": user.get("listed_count", 0),
            "is_verified": user.get("is_verified", False),
            "is_blue_verified": user.get("is_blue_verified", False),
            "profile_pic_url": user.get("profile_pic_url", ""),
            "profile_banner_url": user.get("profile_banner_url", ""),
            "description": user.get("description", ""),
            "external_url": user.get("external_url", ""),
            "number_of_tweets": user.get("number_of_tweets", 0),
            "location": user.get("location", ""),
        }

        # Combine tweet and user information
        tweet_info["user_info"] = user_info
        rich_data.append(tweet_info)

    return rich_data


In [25]:
result = extract_rich_data(output_dict)
print(json.dumps(result, indent=4))

[
    {
        "tweet_id": "1931002435889254479",
        "creation_date": "Fri Jun 06 14:57:11 +0000 2025",
        "text": "On Monday, we sent a letter to Apple CEO Tim Cook, demanding an apology and the removal of a scene from one of its shows that involves \"Eucharistic desecration.\" https://t.co/Gf137tSqoS",
        "language": "en",
        "favorite_count": 2252,
        "retweet_count": 406,
        "reply_count": 47,
        "quote_count": 22,
        "views": 40009,
        "video_view_count": null,
        "source": "Twitter for iPhone",
        "conversation_id": "1931002435889254479",
        "in_reply_to_status_id": null,
        "quoted_status_id": null,
        "expanded_url": "https://x.com/CatholicVote/status/1931002435889254479/photo/1",
        "media": [
            {
                "type": "photo",
                "url": "https://pbs.twimg.com/media/GsxMezyWcAAv2xs.jpg",
                "display_url": "pic.x.com/Gf137tSqoS",
                "expanded_url": "htt

# Twitter API 

In [44]:
import json
from datetime import datetime, timedelta
conn = http.client.HTTPSConnection("twitter135.p.rapidapi.com")
headers = {
    'x-rapidapi-key': "7ee1c65373msh181b4a01f21d239p1e5ff7jsnc2f07e071e6d",
    'x-rapidapi-host': "twitter135.p.rapidapi.com"
}
company_name = "apple"
job_title = "software engineer"
limit = 5
date = datetime.today()
start_date = (date.today() - timedelta(days=30)).isoformat(),
end_date = date.today().isoformat()
query_params = {
    'q': f'("{company_name}" OR "company_name inc") OR ("ceo of {company_name}" OR "{company_name} ceo") OR ("{job_title}" AND ("{company_name}" OR "{company_name} inc")) -filter:retweets lang:en',
    'count': limit,
    'type': 'Top',
    'safe_search':'true'
}
# get url safe query
url_safe_query = urllib.parse.urlencode(query_params)
url_safe_query

'q=%28%22apple%22+OR+%22apple+inc%22%29+OR+%28%22ceo+of+apple%22+OR+%22apple+ceo%22%29+OR+%28%22software+engineer%22+AND+%28%22apple%22+OR+%22apple+inc%22%29%29+-filter%3Aretweets+lang%3Aen&count=5&type=Top&safe_search=true'

In [36]:
conn.request("GET", f"/Search/?{url_safe_query}", headers=headers)

In [37]:
res = conn.getresponse()
data = res.read()

print(data.decode("utf-8"))

{"data":{"search_by_raw_query":{"search_timeline":{"timeline":{"instructions":[{"type":"TimelineAddEntries","entries":[{"entryId":"toptabsrpusermodule-1935236276844232704","sortIndex":"1935236276844232704","content":{"entryType":"TimelineTimelineModule","__typename":"TimelineTimelineModule","items":[{"entryId":"toptabsrpusermodule-1935236276844232704-user-1636590253","item":{"itemContent":{"itemType":"TimelineUser","__typename":"TimelineUser","user_results":{"result":{"__typename":"User","id":"VXNlcjoxNjM2NTkwMjUz","rest_id":"1636590253","affiliates_highlighted_label":{"label":{"url":{"url":"https://twitter.com/Apple","urlType":"DeepLink"},"badge":{"url":"https://pbs.twimg.com/profile_images/1892248257516224513/SzZdRSkx_bigger.png"},"description":"Apple","userLabelType":"BusinessLabel","userLabelDisplayType":"Badge"}},"has_graduated_access":true,"is_blue_verified":true,"legacy":{"can_dm":false,"can_media_tag":true,"created_at":"Wed Jul 31 22:41:25 +0000 2013","default_profile":false,"d

In [38]:
od = json.loads(data)

In [39]:
od

{'data': {'search_by_raw_query': {'search_timeline': {'timeline': {'instructions': [{'type': 'TimelineAddEntries',
       'entries': [{'entryId': 'toptabsrpusermodule-1935236276844232704',
         'sortIndex': '1935236276844232704',
         'content': {'entryType': 'TimelineTimelineModule',
          '__typename': 'TimelineTimelineModule',
          'items': [{'entryId': 'toptabsrpusermodule-1935236276844232704-user-1636590253',
            'item': {'itemContent': {'itemType': 'TimelineUser',
              '__typename': 'TimelineUser',
              'user_results': {'result': {'__typename': 'User',
                'id': 'VXNlcjoxNjM2NTkwMjUz',
                'rest_id': '1636590253',
                'affiliates_highlighted_label': {'label': {'url': {'url': 'https://twitter.com/Apple',
                   'urlType': 'DeepLink'},
                  'badge': {'url': 'https://pbs.twimg.com/profile_images/1892248257516224513/SzZdRSkx_bigger.png'},
                  'description': 'Apple',
 

In [42]:
def extract_tweets_with_users(response_dict):
    results = []
    instructions = (
        response_dict.get('data', {})
        .get('search_by_raw_query', {})
        .get('search_timeline', {})
        .get('timeline', {})
        .get('instructions', [])
    )
    for instruction in instructions:
        if instruction.get('type') == 'TimelineAddEntries':
            for entry in instruction.get('entries', []):
                content = entry.get('content', {})
                if content.get('entryType') == 'TimelineTimelineItem':
                    item_content = content.get('itemContent', {})
                    if item_content.get('itemType') == 'TimelineTweet':
                        tweet_result = item_content.get('tweet_results', {}).get('result', {})
                        tweet_legacy = tweet_result.get('legacy', {})
                        tweet_core_user = tweet_result.get('core', {}).get('user_results', {}).get('result', {})
                        user_legacy = tweet_core_user.get('legacy', {})
                        user_info = {
                            "user_id": tweet_core_user.get('rest_id', ''),
                            "screen_name": user_legacy.get('screen_name', ''),
                            "name": user_legacy.get('name', ''),
                            "description": user_legacy.get('description', ''),
                            "created_at": user_legacy.get('created_at', ''),
                            "followers_count": user_legacy.get('followers_count', 0),
                            "friends_count": user_legacy.get('friends_count', 0),
                            "statuses_count": user_legacy.get('statuses_count', 0),
                            "verified": tweet_core_user.get('verification', {}).get('verified', False),
                            "is_blue_verified": tweet_core_user.get('is_blue_verified', False),
                            "profile_image_url": user_legacy.get('profile_image_url_https', ''),
                            "profile_banner_url": user_legacy.get('profile_banner_url', ''),
                            "location": tweet_core_user.get('location', {}).get('location', ''),
                            "protected": tweet_core_user.get('privacy', {}).get('protected', False),
                        }
                        tweet_info = {
                            "tweet_id": tweet_result.get('rest_id', ''),
                            "text": tweet_legacy.get('full_text', ''),
                            "created_at": tweet_legacy.get('created_at', ''),
                            "hashtags": [h.get('text', '') for h in tweet_legacy.get('entities', {}).get('hashtags', [])],
                            "mentions": [m.get('screen_name', '') for m in tweet_legacy.get('entities', {}).get('user_mentions', [])],
                            "urls": [u.get('expanded_url', '') for u in tweet_legacy.get('entities', {}).get('urls', [])],
                            "media": [
                                {
                                    "type": m.get("type", ""),
                                    "media_url": m.get("media_url_https", ""),
                                    "video_info": m.get("video_info", None),
                                    "sizes": m.get("sizes", None)
                                }
                                for m in tweet_legacy.get('entities', {}).get('media', [])
                            ],
                            "retweet_count": tweet_legacy.get('retweet_count', 0),
                            "favorite_count": tweet_legacy.get('favorite_count', 0),
                            "lang": tweet_legacy.get('lang', ''),
                            "source": tweet_result.get('source', ''),
                            "is_quote_status": tweet_legacy.get('is_quote_status', False),
                            "quote_count": tweet_legacy.get('quote_count', 0),
                            "reply_count": tweet_legacy.get('reply_count', 0),
                        }
                        results.append({"user_info": user_info, "tweet_info": tweet_info})
    return results

In [43]:
extract_tweets_with_users(od)

[{'user_info': {'user_id': '18080620',
   'screen_name': 'CatholicVote',
   'name': 'CatholicVote',
   'description': 'CatholicVote is a community of patriotic Americans who believe our nation’s founding principles are good and true, and worth fighting for.',
   'created_at': 'Fri Dec 12 17:22:36 +0000 2008',
   'followers_count': 89613,
   'friends_count': 2413,
   'statuses_count': 26672,
   'verified': False,
   'is_blue_verified': False,
   'profile_image_url': 'https://pbs.twimg.com/profile_images/796404920392028160/T7ohIMTJ_normal.jpg',
   'profile_banner_url': 'https://pbs.twimg.com/profile_banners/18080620/1670947551',
   'location': '',
   'protected': False},
  'tweet_info': {'tweet_id': '1931002435889254479',
   'text': 'On Monday, we sent a letter to Apple CEO Tim Cook, demanding an apology and the removal of a scene from one of its shows that involves "Eucharistic desecration." https://t.co/Gf137tSqoS',
   'created_at': 'Fri Jun 06 14:57:11 +0000 2025',
   'hashtags': [],
